In [11]:
import pandas as pd
import numpy as np
from itertools import combinations
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error
import shap

# Load data
stint_data = pd.read_csv('stint_data.csv')
player_data = pd.read_csv('player_data.csv')

In [12]:
# Calculate +/- for each team
team_plus_minus = {}

for _, row in stint_data.iterrows():
    h_team = row['h_team']
    a_team = row['a_team']
    goal_diff = row['h_goals'] - row['a_goals']
    
    if h_team not in team_plus_minus:
        team_plus_minus[h_team] = 0
    team_plus_minus[h_team] += goal_diff
    
    if a_team not in team_plus_minus:
        team_plus_minus[a_team] = 0
    team_plus_minus[a_team] -= goal_diff

team_plus_minus_df = pd.DataFrame(list(team_plus_minus.items()), 
                                   columns=['team', 'plus_minus'])
team_plus_minus_df = team_plus_minus_df.sort_values('plus_minus', ascending=False).reset_index(drop=True)

In [13]:
# Calculate +/- for each player
player_plus_minus = {}

for _, row in stint_data.iterrows():
    goal_diff = row['h_goals'] - row['a_goals']
    
    # Home players get positive goal differential
    for col in ['home1', 'home2', 'home3', 'home4']:
        player = row[col]
        if player not in player_plus_minus:
            player_plus_minus[player] = 0
        player_plus_minus[player] += goal_diff
    
    # Away players get negative goal differential
    for col in ['away1', 'away2', 'away3', 'away4']:
        player = row[col]
        if player not in player_plus_minus:
            player_plus_minus[player] = 0
        player_plus_minus[player] -= goal_diff

plus_minus_df = pd.DataFrame(list(player_plus_minus.items()), 
                              columns=['player', 'plus_minus'])
plus_minus_df = plus_minus_df.sort_values('plus_minus', ascending=False).reset_index(drop=True)

In [14]:
# Calculate +/- per 10 minutes for each player
player_minutes = {}

for _, row in stint_data.iterrows():
    minutes = row['minutes']
    
    for col in ['home1', 'home2', 'home3', 'home4']:
        player = row[col]
        if player not in player_minutes:
            player_minutes[player] = 0
        player_minutes[player] += minutes
    
    for col in ['away1', 'away2', 'away3', 'away4']:
        player = row[col]
        if player not in player_minutes:
            player_minutes[player] = 0
        player_minutes[player] += minutes

player_minutes_df = pd.DataFrame(list(player_minutes.items()), 
                                  columns=['player', 'total_minutes'])

player_stats = plus_minus_df.merge(player_minutes_df, on='player')
player_stats['plus_minus_per_10min'] = (player_stats['plus_minus'] / player_stats['total_minutes']) * 10
player_stats = player_stats.sort_values('plus_minus_per_10min', ascending=False).reset_index(drop=True)

In [15]:
# Calculate +/- per 10 minutes for each team
team_minutes = {}

for _, row in stint_data.iterrows():
    h_team = row['h_team']
    a_team = row['a_team']
    minutes = row['minutes']
    
    if h_team not in team_minutes:
        team_minutes[h_team] = 0
    team_minutes[h_team] += minutes
    
    if a_team not in team_minutes:
        team_minutes[a_team] = 0
    team_minutes[a_team] += minutes

team_minutes_df = pd.DataFrame(list(team_minutes.items()), 
                                columns=['team', 'total_minutes'])

team_stats = team_plus_minus_df.merge(team_minutes_df, on='team')
team_stats['plus_minus_per_10min'] = (team_stats['plus_minus'] / team_stats['total_minutes']) * 10
team_stats = team_stats.sort_values('plus_minus_per_10min', ascending=False).reset_index(drop=True)

In [17]:
# Regularized Lineup Impact Model (RAPM-style) per team

def create_lineup_id(players):
    return tuple(sorted(players))

team_lineups = {}
lineup_data = []

for idx, row in stint_data.iterrows():
    home_team = row['h_team']
    home_lineup = create_lineup_id([row['home1'], row['home2'], row['home3'], row['home4']])
    home_goal_diff = row['h_goals'] - row['a_goals']
    
    if home_team not in team_lineups:
        team_lineups[home_team] = set()
    team_lineups[home_team].add(home_lineup)
    
    lineup_data.append({
        'team': home_team,
        'lineup': home_lineup,
        'goal_diff': home_goal_diff,
        'minutes': row['minutes'],
        'side': 'home'
    })
    
    away_team = row['a_team']
    away_lineup = create_lineup_id([row['away1'], row['away2'], row['away3'], row['away4']])
    away_goal_diff = row['a_goals'] - row['h_goals']
    
    if away_team not in team_lineups:
        team_lineups[away_team] = set()
    team_lineups[away_team].add(away_lineup)
    
    lineup_data.append({
        'team': away_team,
        'lineup': away_lineup,
        'goal_diff': away_goal_diff,
        'minutes': row['minutes'],
        'side': 'away'
    })

lineup_df = pd.DataFrame(lineup_data)

team_lineup_models = {}

for team in sorted(team_lineups.keys()):
    team_data = lineup_df[lineup_df['team'] == team].copy()
    unique_lineups = sorted(team_lineups[team])
    
    lineup_to_idx = {lineup: idx for idx, lineup in enumerate(unique_lineups)}
    
    X_lineup = np.zeros((len(team_data), len(unique_lineups)))
    y_lineup = []
    
    for i, (_, row) in enumerate(team_data.iterrows()):
        lineup_idx = lineup_to_idx[row['lineup']]
        X_lineup[i, lineup_idx] = 1
        y_lineup.append(row['goal_diff'])
    
    y_lineup = np.array(y_lineup)
    
    alpha = 50.0
    lineup_model = Ridge(alpha=alpha)
    lineup_model.fit(X_lineup, y_lineup)
    
    lineup_impacts = pd.DataFrame({
        'lineup': unique_lineups,
        'impact': lineup_model.coef_
    })
    
    lineup_stats = team_data.groupby('lineup').agg({
        'goal_diff': ['sum', 'mean'],
        'minutes': ['sum', 'count']
    }).reset_index()
    lineup_stats.columns = ['lineup', 'total_goal_diff', 'avg_goal_diff', 'total_minutes', 'num_stints']
    
    lineup_impacts = lineup_impacts.merge(lineup_stats, on='lineup')
    lineup_impacts['impact_per_10min'] = (lineup_impacts['impact'] / lineup_impacts['total_minutes']) * 10
    lineup_impacts = lineup_impacts.sort_values('impact', ascending=False).reset_index(drop=True)
    lineup_impacts['lineup_str'] = lineup_impacts['lineup'].apply(lambda x: ', '.join(x))
    
    team_lineup_models[team] = {
        'model': lineup_model,
        'impacts': lineup_impacts,
        'r2': lineup_model.score(X_lineup, y_lineup)
    }

In [ ]:
# Team-specific Player Value Models with Enhanced Features

rating_lookup = player_data.set_index('player')['rating'].to_dict()
team_strength = team_stats.set_index('team')['plus_minus_per_10min'].to_dict()
all_teams = sorted(stint_data['h_team'].unique())

team_player_models = {}

for team in all_teams:
    team_player_list = player_data[player_data['player'].str.startswith(f"{team}_")]['player'].tolist()
    
    if len(team_player_list) == 0:
        continue
    
    player_to_idx = {player: idx for idx, player in enumerate(team_player_list)}
    
    X_features = []
    y_values = []
    
    for _, row in stint_data.iterrows():
        is_home = row['h_team'] == team
        is_away = row['a_team'] == team
        
        if not (is_home or is_away):
            continue
            
        if is_home:
            goal_diff = row['h_goals'] - row['a_goals']
            our_players = [row['home1'], row['home2'], row['home3'], row['home4']]
            opp_players = [row['away1'], row['away2'], row['away3'], row['away4']]
            opp_team = row['a_team']
        else:
            goal_diff = row['a_goals'] - row['h_goals']
            our_players = [row['away1'], row['away2'], row['away3'], row['away4']]
            opp_players = [row['home1'], row['home2'], row['home3'], row['home4']]
            opp_team = row['h_team']
        
        player_indicators = np.zeros(len(team_player_list))
        player_ratings = np.zeros(len(team_player_list))
        
        for p in our_players:
            if p in player_to_idx:
                idx = player_to_idx[p]
                player_indicators[idx] = 1
                player_ratings[idx] = rating_lookup.get(p, 0)
        
        our_total_rating = sum([rating_lookup.get(p, 0) for p in our_players])
        our_avg_rating = our_total_rating / 4.0
        
        opp_total_rating = sum([rating_lookup.get(p, 0) for p in opp_players])
        opp_avg_rating = opp_total_rating / 4.0
        opp_strength = team_strength.get(opp_team, 0)
        
        stint_minutes = row['minutes']
        home_advantage = 1.0 if is_home else 0.0
        
        rating_diff = our_avg_rating - opp_avg_rating
        rating_product = our_avg_rating * opp_avg_rating
        
        features = np.concatenate([
            player_indicators,
            player_ratings,
            [our_total_rating],
            [our_avg_rating],
            [opp_total_rating],
            [opp_avg_rating],
            [opp_strength],
            [rating_diff],
            [rating_product],
            [stint_minutes],
            [home_advantage]
        ])
        
        X_features.append(features)
        y_values.append(goal_diff)
    
    if len(X_features) == 0:
        continue
    
    feature_names = []
    
    for player in team_player_list:
        feature_names.append(f"indicator_{player}")
    
    for player in team_player_list:
        feature_names.append(f"rating_{player}")
    
    feature_names.extend([
        'our_total_rating',
        'our_avg_rating',
        'opp_total_rating',
        'opp_avg_rating',
        'opp_strength',
        'rating_diff',
        'rating_product',
        'stint_minutes',
        'home_advantage'
    ])
    
    X = pd.DataFrame(X_features, columns=feature_names)
    y = np.array(y_values)
    
    models_to_try = {
        'Ridge (α=10)': Ridge(alpha=10.0),
        'Lasso (α=1)': Lasso(alpha=1.0, max_iter=10000),
        'Random Forest': RandomForestRegressor(n_estimators=50, max_depth=8, random_state=42),
        'Gradient Boost': GradientBoostingRegressor(n_estimators=50, max_depth=5, random_state=42)
    }
    
    best_r2 = -float('inf')
    best_model_name = None
    best_model = None
    all_model_r2 = {}  # Track R² for all models
    
    for name, model in models_to_try.items():
        model.fit(X, y)
        y_pred = model.predict(X)
        r2 = r2_score(y, y_pred)
        all_model_r2[name] = r2  # Store R² for this model
        
        if r2 > best_r2:
            best_r2 = r2
            best_model_name = name
            best_model = model
    
    if hasattr(best_model, 'coef_'):
        player_values = best_model.coef_[:len(team_player_list)]
        rating_effects = best_model.coef_[len(team_player_list):2*len(team_player_list)]
    else:
        explainer = shap.TreeExplainer(best_model)
        shap_values = explainer.shap_values(X)
        
        player_values = np.mean(shap_values[:, :len(team_player_list)], axis=0)
        rating_effects = np.mean(shap_values[:, len(team_player_list):2*len(team_player_list)], axis=0)
    
    results_df = pd.DataFrame({
        'player': team_player_list,
        'player_value': player_values,
        'rating_effect': rating_effects
    })
    
    results_df['rating'] = results_df['player'].map(rating_lookup)
    
    # Normalize player_value and rating_effect to [-1, 1]
    for col in ['player_value', 'rating_effect']:
        min_val = results_df[col].min()
        max_val = results_df[col].max()
        if max_val != min_val:  # Avoid division by zero
            results_df[col] = 2 * (results_df[col] - min_val) / (max_val - min_val) - 1
        else:
            results_df[col] = 0  # If all values are the same, set to 0
    
    # Combined impact is the sum of normalized values
    results_df['combined_impact'] = results_df['player_value'] + results_df['rating_effect']
    
    results_df = results_df.sort_values('player_value', ascending=False).reset_index(drop=True)
    
    feature_df = X.copy()
    feature_df['goal_differential'] = y
    
    team_player_models[team] = {
        'model': best_model,
        'model_name': best_model_name,
        'results': results_df,
        'feature_df': feature_df,
        'r2': best_r2,
        'rmse': np.sqrt(mean_squared_error(y, best_model.predict(X))),
        'all_model_r2': all_model_r2  # Store all model R² scores
    }


In [19]:
results_df

,player,player_value,rating_effect,rating,combined_impact
0,USA_p9,1.000000,0.616338,1.0,1.000000
1,USA_p1,0.562722,0.957927,3.0,0.737741
2,USA_p7,0.484234,0.961810,1.5,0.667661
3,USA_p12,0.477533,0.374546,3.0,0.426285
4,USA_p5,0.439913,0.673195,2.0,0.511589
5,USA_p2,0.432608,0.334033,3.0,0.369052
6,USA_p6,0.384330,0.448422,0.5,0.370814
7,USA_p11,0.362050,0.000000,3.5,0.170840
8,USA_p8,0.250906,0.075009,2.5,0.099449
9,USA_p3,0.204817,0.179684,3.5,0.099316


In [20]:
feature_df

,indicator_USA_p1,indicator_USA_p2,indicator_USA_p3,indicator_USA_p4,indicator_USA_p5,indicator_USA_p6,indicator_USA_p7,indicator_USA_p8,indicator_USA_p9,indicator_USA_p10,...,our_total_rating,our_avg_rating,opp_total_rating,opp_avg_rating,opp_strength,rating_diff,rating_product,stint_minutes,home_advantage,goal_differential
0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,7.0,1.750,7.5,1.875,4.779162,-0.125,3.281250,4.252969,1.0,-5
1,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,7.0,1.750,7.5,1.875,4.779162,-0.125,3.281250,5.688809,1.0,-5
2,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,7.5,1.875,7.0,1.750,4.779162,0.125,3.281250,1.149557,1.0,-1
3,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,7.5,1.875,7.5,1.875,4.779162,0.000,3.515625,3.511617,1.0,2
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,...,7.0,1.750,7.0,1.750,4.779162,0.000,3.062500,2.163139,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,7.5,1.875,7.0,1.750,-3.575672,0.125,3.281250,3.812630,0.0,3
1225,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,7.5,1.875,7.5,1.875,-3.575672,0.000,3.515625,6.339196,0.0,12
1226,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,8.0,2.000,7.0,1.750,-3.575672,0.250,3.500000,3.603345,0.0,7
1227,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,8.0,2.000,7.0,1.750,-3.575672,0.250,3.500000,4.062071,0.0,8


In [21]:
# Optimal 4-Player Lineup Selection

optimal_lineups = {}

for team in sorted(team_player_models.keys()):
    results = team_player_models[team]['results']
    
    players_list = []
    for _, row in results.iterrows():
        players_list.append({
            'player': row['player'],
            'value': row['player_value'],
            'rating': row['rating'],
            'combined_impact': row['combined_impact']
        })
    
    all_combinations = list(combinations(players_list, 4))
    valid_total_ratings = [7.0, 7.5, 8.0]
    
    best_lineup = None
    best_total_value = -float('inf')
    
    lineup_evaluations = []
    valid_lineups_count = 0
    
    for combo in all_combinations:
        total_value = sum([p['value'] for p in combo])
        total_rating = sum([p['rating'] for p in combo])
        avg_rating = total_rating / 4.0
        total_combined = sum([p['combined_impact'] for p in combo])
        
        if total_rating not in valid_total_ratings:
            continue
        
        valid_lineups_count += 1
        
        lineup_evaluations.append({
            'players': [p['player'] for p in combo],
            'total_value': total_value,
            'total_rating': total_rating,
            'avg_rating': avg_rating,
            'total_combined_impact': total_combined
        })
        
        if total_value > best_total_value:
            best_total_value = total_value
            best_lineup = combo
    
    if best_lineup is None:
        continue
    
    optimal_lineups[team] = {
        'best_lineup': best_lineup,
        'best_total_value': best_total_value,
        'all_evaluations': lineup_evaluations
    }

In [22]:
all_model_r2

{'Ridge (α=10)': 0.16983890071010166,
 'Lasso (α=1)': 0.06558121328755961,
 'Random Forest': 0.6123989552861241,
 'Gradient Boost': 0.6406407344271676}